# Build file creation of cuboids <img align="right" width="192" height="64" src="https://freemelt.com/app/uploads/freemeltLogo-1.png">

In Electron Beam Manufacturing, it's common to expedite development by simultaneously building multiple objects. This is commonly done by arranging cuboids in a grid and line melting them individually. This second part of the five-part guide will show how this can be conveniently achieved using a class. 

Importing the necessary libraries:

In [106]:
import obplib as obp
import os
import interactive_viewer as iw
import random as rand

Using classes to place build objects simplifies the code and improves readability by centralizing related operations. Additionally, it provides a more intuitive interface for users to interact with, as they can simply create an instance of the class and call the desired methods. From this part onwards, classes will be the central object around which the code is centered. 

Here, the `Cuboid` class is defined. It stores the attributes `midpoint`, `dimensions` as two-dimensional lists, and `speed`, `spot_size`, `power` as ints.

In [107]:
class Cuboid:
    def __init__(self, midpoint, dimensions, speed, spot_size, power):
        self.midpoint = midpoint
        self.dimensions = dimensions
        self.speed = speed
        self.spot_size = spot_size
        self.power = power

A point grid is established, with `density` determining the number of points per micrometer.

In [108]:
%%add_to Cuboid
def _place_points(self, density):
    number_of_points = [int(self.dimensions[0] * density), int(self.dimensions[1] * density)]
    points = [
                [
                obp.Point(j / density + self.midpoint[0] - self.dimensions[0] / 2, i / density + self.midpoint[1] - self.dimensions[1] / 2) 
                for j in range(number_of_points[0] + 1)
                ]
                for i in range(number_of_points[1] + 1)
    ]
    return [ele for ele in points if ele] #return points within the build area

## Example - four `Cuboid` objects as `Point`'s
In this example, four `Cuboid` objects are initialized and placed in a 2 $\times$ 2 grid. 
The density is set to be much lower than the `Cuboid` dimensions, so there will only be one point per `Cuboid`.

In [114]:
rec_numbers = [2, 2]
dimensions = [100, 150]
density = 0.001 #low number
speed, spot_size, power = 100, 100, 100
midpoints = [
        [
            (j - 1/4) * dimensions[0] * 2,
            (i - 1/4) * dimensions[1] * 2
        ] 
        for i in range(rec_numbers[0]) for j in range(rec_numbers[1])
    ]

cuboids = [Cube(midpoints[i], dimensions, speed, spot_size, power) for i in range(rec_numbers[0] * rec_numbers[1])]

points = []
for i in range(rec_numbers[0] * rec_numbers[1]):
    points = points + cuboids[i]._place_points(density)

for i in range(rec_numbers[0] * rec_numbers[1]):
    for j in range(len(points[i])):
        print("p_{}.x = {}, p_{}.y = {}".format(i, points[i][j].x, i, points[i][j].y))

p_0.x = -100.0, p_0.y = -150.0
p_1.x = 100.0, p_1.y = -150.0
p_2.x = -100.0, p_2.y = 150.0
p_3.x = 100.0, p_3.y = 150.0


Unidirectional line melting patterns is common within EBM due to their effectiveness and ease of implementation. To achieve this, the method `place_lines_one_direction` is introduced. 

In [110]:
%%add_to Cuboid
def place_lines_one_direction(self, density, filename, reverse):
    # Generate the points on the powder bed
    points = self._place_points(density)

    # Create the lines based on the points
    lines = []
    y_range = range(len(points))
    bp = obp.Beamparameters(self.spot_size, self.power)
    for y in y_range:
        lines.append(obp.Line(points[y][0], points[y][-1], self.speed, bp))

    # Write the lines to the file
    try:
        old_obp = obp.read_obp(filename)
        new_obp = old_obp + lines
        obp.write_obp(new_obp, filename)
    except:
        obp.write_obp(lines, filename)
        
    if reverse == True:
        lines = [obp.Line(line.P2, line.P1, line.Speed, line.bp) for line in reversed(lines)]

    return lines

In [111]:
%%add_to Cuboid
#calls the melt pattern on each island in a random order
def melt_cuboid(self, density, filename, reverse = False):
    lines = Cuboid.place_lines_one_direction(density, filename, reverse) #change for different melting pattern
    return lines

## Example - 3 $\times$ 3 cuboids

In [112]:
#code example 
filename = "hej3.obp"
try:    
    os.remove(filename)
except:
    pass

density = 0.1
rec_numbers = [3, 3]
dimensions = [100, 100]
midpoint_list = [
    [
        (i + 1 - rec_numbers[0]/2) * dimensions[0] * 2, 
        (j + 1 - rec_numbers[1]/2) * dimensions[1] * 2
    ] 
    for i in range(rec_numbers[0]) for j in range(rec_numbers[1])
]
cuboid_list = [
    Cuboid(midpoint = midpoint_list[i], dimensions=[100, 100], speed=100, spot_size=100, power=100)
        for i in range(rec_numbers[0] * rec_numbers[1])
]
lines = []
for i in range(rec_numbers[0] * rec_numbers[1]):
    lines = lines + cuboid_list[i].place_lines_one_direction(density, filename, False)

iw.interactive_viewer(lines)

interactive(children=(IntText(value=99, description='Max Lines:'), IntSlider(value=50, description='upper_lim'…